In [13]:
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all modules (except those excluded by %aimport) every time before executing the Python code typed:
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# General imports
from pathlib import Path
import numpy as np
import xarray as xr
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20, 10)
plt.rcParams["figure.facecolor"] = "white"

# power_perceiver imports
from power_perceiver.load_prepared_batches.data_loader import HRVSatellite, GSP, PV, Sun
from power_perceiver.load_prepared_batches.prepared_dataset import PreparedDataset
from power_perceiver.transforms.pv import PVPowerRollingWindow
from power_perceiver.transforms.satellite import PatchSatellite
from power_perceiver.xr_batch_processor import ReduceNumPVSystems, SelectPVSystemsNearCenterOfImage, AlignGSPTo5Min
from power_perceiver.np_batch_processor import EncodeSpaceTime, Topography
from power_perceiver.consts import BatchKey



In [15]:
DATA_PATH = Path("~/dev/ocf/power_perceiver/data_for_testing/").expanduser()
assert DATA_PATH.exists()

In [17]:
BATCH_IDX = 0
gsp = GSP(data_path=DATA_PATH)[BATCH_IDX]
hrv = HRVSatellite(data_path=DATA_PATH)[BATCH_IDX]
pv = PV(data_path=DATA_PATH)[BATCH_IDX]

In [18]:
gsp

<xarray.Dataset>
Dimensions:           (example: 32, time: 6)
Coordinates:
    gsp_id            (example) int32 332 120 64 70 324 113 ... 32 28 43 332 129
    time_utc          (example, time) datetime64[ns] 2020-09-12T09:00:00 ... ...
    x_osgb            (example) float32 2.22e+05 1.828e+05 ... 4.532e+05
    y_osgb            (example) float32 7.157e+05 5.16e+04 ... 7.157e+05 3.9e+05
Dimensions without coordinates: example, time
Data variables:
    capacity_mwp      (example, time) float32 0.3418 0.3418 ... 17.13 17.13
    power_mw          (example, time) float32 0.006855 0.06283 ... 7.415 5.45
    power_normalised  (example, time) float32 0.02005 0.1838 ... 0.433 0.3182

In [19]:
hrv

<xarray.Dataset>
Dimensions:          (example: 32, channel: 1, time: 31, y: 64, x: 64)
Coordinates:
    channel_name     (example, channel) object 'HRV' 'HRV' 'HRV' ... 'HRV' 'HRV'
    time_utc         (example, time) datetime64[ns] 2020-09-12T09:05:00 ... 2...
    x_geostationary  (example, x) float32 -8.461e+05 -8.451e+05 ... -6.281e+05
    x_osgb           (example, y, x) float32 2.007e+05 2.018e+05 ... 4.753e+05
    y_geostationary  (example, y) float32 4.818e+06 4.819e+06 ... 4.75e+06
    y_osgb           (example, y, x) float32 6.382e+05 6.381e+05 ... 4.567e+05
Dimensions without coordinates: example, channel, time, y, x
Data variables:
    data             (example, time, channel, y, x) int16 260 260 ... 139 138

In [20]:
pv

<xarray.Dataset>
Dimensions:               (example: 32, pv_system: 128, time: 31)
Coordinates:
    pv_system_id          (example, pv_system) float64 2.766e+03 ... nan
    pv_system_row_number  (example, pv_system) int64 7 52 59 74 87 ... 0 0 0 0 0
    time_utc              (example, time) datetime64[ns] 2020-09-12T09:05:00 ...
    x_osgb                (example, pv_system) float32 2.208e+05 ... nan
    y_osgb                (example, pv_system) float32 6.608e+05 ... nan
Dimensions without coordinates: example, pv_system, time
Data variables:
    capacity_wp           (example, pv_system) float32 2.176e+03 3e+03 ... nan
    power_w               (example, time, pv_system) float32 246.5 811.9 ... nan
    power_normalised      (example, time, pv_system) float32 0.1133 ... nan
    pv_mask               (example, pv_system) bool True True ... False False

In [21]:
31-24

7

In [95]:
# Find the corresponding GSP 30 minute timestep for each 5 minute satellite timestep.
# We do this by taking the `ceil("30T")` of each 5 minute satellite timestep.
# Most of the code below is just converting from xarray to Pandas and back
# so we can use `pd.DatetimeIndex.ceil` on each datetime:
time_5_min_series = hrv.time_utc.to_series()
time_5_min_dt_index = pd.DatetimeIndex(time_5_min_series)
time_30_min_dt_index = time_5_min_dt_index.ceil("30T")
time_30_min_series = pd.Series(
    time_30_min_dt_index,
    index=time_5_min_series.index
)
time_30_min_da = time_30_min_series.to_xarray()

In [112]:
# Loop through each example and find the index into the GSP time dimension
# of the GSP timestep corresponding to each 5 minute satellite timestep:
gsp_5_min_for_all_examples = []
max_time_idx = len(gsp.time) - 1
for example_i in gsp.example:
    idx_into_gsp = np.searchsorted(
        gsp.sel(example=example_i).time_utc.values,
        time_30_min_da.sel(example=example_i).values,
    )
    gsp_5_min = gsp.isel(example=example_i, time=idx_into_gsp.clip(max=max_time_idx))
    
    # Now, for any timestep where we don't have GSP data, set to NaN:
    mask = idx_into_gsp <= max_time_idx
    gsp_5_min = gsp_5_min.where(mask)
    gsp_5_min["time_utc"] = gsp_5_min.time_utc.where(mask)
    gsp_5_min_for_all_examples.append(gsp_5_min)
    
gsp_5_min_for_all_examples = xr.concat(gsp_5_min_for_all_examples, dim="example")
gsp_5_min_for_all_examples

<xarray.Dataset>
Dimensions:           (example: 32, time: 31)
Coordinates:
    gsp_id            (example) int32 332 120 64 70 324 113 ... 32 28 43 332 129
    time_utc          (example, time) datetime64[ns] 2020-09-12T09:30:00 ... NaT
    x_osgb            (example) float32 2.22e+05 1.828e+05 ... 4.532e+05
    y_osgb            (example) float32 7.157e+05 5.16e+04 ... 7.157e+05 3.9e+05
Dimensions without coordinates: example, time
Data variables:
    capacity_mwp      (example, time) float32 0.3418 0.3418 0.3418 ... nan nan
    power_mw          (example, time) float32 0.06283 0.06283 ... nan nan
    power_normalised  (example, time) float32 0.1838 0.1838 0.1838 ... nan nan

In [98]:
time_30_min_da.sel(example=example_i)

<xarray.DataArray 'time_utc' (time: 31)>
array(['2020-09-12T09:30:00.000000000', '2020-09-12T09:30:00.000000000',
       '2020-09-12T09:30:00.000000000', '2020-09-12T09:30:00.000000000',
       '2020-09-12T09:30:00.000000000', '2020-09-12T09:30:00.000000000',
       '2020-09-12T10:00:00.000000000', '2020-09-12T10:00:00.000000000',
       '2020-09-12T10:00:00.000000000', '2020-09-12T10:00:00.000000000',
       '2020-09-12T10:00:00.000000000', '2020-09-12T10:00:00.000000000',
       '2020-09-12T10:30:00.000000000', '2020-09-12T10:30:00.000000000',
       '2020-09-12T10:30:00.000000000', '2020-09-12T10:30:00.000000000',
       '2020-09-12T10:30:00.000000000', '2020-09-12T10:30:00.000000000',
       '2020-09-12T11:00:00.000000000', '2020-09-12T11:00:00.000000000',
       '2020-09-12T11:00:00.000000000', '2020-09-12T11:00:00.000000000',
       '2020-09-12T11:00:00.000000000', '2020-09-12T11:00:00.000000000',
       '2020-09-12T11:30:00.000000000', '2020-09-12T11:30:00.000000000',
       '2020-09-12T11:30:00.000000000', '2020-09-12T11:30:00.000000000',
       '2020-09-12T11:30:00.000000000', '2020-09-12T11:30:00.000000000',
       '2020-09-12T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
    example  int64 0
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 ... 21 22 23 24 25 26 27 28 29 30

In [94]:
hrv.time_utc.sel(example=example_i)

<xarray.DataArray 'time_utc' (time: 31)>
array(['2020-09-12T09:05:00.000000000', '2020-09-12T09:10:00.000000000',
       '2020-09-12T09:15:00.000000000', '2020-09-12T09:20:00.000000000',
       '2020-09-12T09:25:00.000000000', '2020-09-12T09:30:00.000000000',
       '2020-09-12T09:35:00.000000000', '2020-09-12T09:40:00.000000000',
       '2020-09-12T09:45:00.000000000', '2020-09-12T09:50:00.000000000',
       '2020-09-12T09:55:00.000000000', '2020-09-12T10:00:00.000000000',
       '2020-09-12T10:05:00.000000000', '2020-09-12T10:10:00.000000000',
       '2020-09-12T10:15:00.000000000', '2020-09-12T10:20:00.000000000',
       '2020-09-12T10:25:00.000000000', '2020-09-12T10:30:00.000000000',
       '2020-09-12T10:35:00.000000000', '2020-09-12T10:40:00.000000000',
       '2020-09-12T10:45:00.000000000', '2020-09-12T10:50:00.000000000',
       '2020-09-12T10:55:00.000000000', '2020-09-12T11:00:00.000000000',
       '2020-09-12T11:05:00.000000000', '2020-09-12T11:10:00.000000000',
       '2020-09-12T11:15:00.000000000', '2020-09-12T11:20:00.000000000',
       '2020-09-12T11:25:00.000000000', '2020-09-12T11:30:00.000000000',
       '2020-09-12T11:35:00.000000000'], dtype='datetime64[ns]')
Coordinates:
    time_utc  (time) datetime64[ns] 2020-09-12T09:05:00 ... 2020-09-12T11:35:00
Dimensions without coordinates: time

In [4]:
dataset = PreparedDataset(
    data_path=DATA_PATH,
    data_loaders=[
        HRVSatellite(
            transforms=[
                PatchSatellite(),
            ]
        ),
        PV(transforms=[PVPowerRollingWindow()]),
        GSP(),
    ],
    xr_batch_processors=[
        SelectPVSystemsNearCenterOfImage(),
        ReduceNumPVSystems(requested_num_pv_systems=8),
        AlignGSPTo5Min(),
    ],
    np_batch_processors=[
        EncodeSpaceTime(),
        Topography("/home/jack/europe_dem_2km_osgb.tif"),
    ],
)


/home/jack/miniconda3/envs/power_perceiver/lib/python3.9/site-packages/pyresample/image.py:59: FutureWarning: Usage of ImageContainer is deprecated, please use NumpyResamplerBilinear class instead
  warnings.warn(
/home/jack/miniconda3/envs/power_perceiver/lib/python3.9/site-packages/pyproj/crs/crs.py:1282: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/jack/miniconda3/envs/power_perceiver/lib/python3.9/site-packages/pyproj/crs/crs.py:1282: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/jack/miniconda3/envs/power_perceiver/lib/python3.9/site-pack

In [5]:
%%time
np_batch = dataset[0]

CPU times: user 627 ms, sys: 12.1 ms, total: 639 ms
Wall time: 638 ms


In [6]:

np_batch.keys()



dict_keys([<BatchKey.hrvsatellite: 1>, <BatchKey.hrvsatellite_time_utc: 6>, <BatchKey.hrvsatellite_y_osgb: 2>, <BatchKey.hrvsatellite_x_osgb: 3>, <BatchKey.hrvsatellite_y_geostationary: 4>, <BatchKey.hrvsatellite_x_geostationary: 5>, <BatchKey.pv: 11>, <BatchKey.pv_system_row_number: 12>, <BatchKey.pv_id: 13>, <BatchKey.pv_capacity_wp: 14>, <BatchKey.pv_mask: 15>, <BatchKey.pv_time_utc: 18>, <BatchKey.pv_x_osgb: 17>, <BatchKey.pv_y_osgb: 16>, <BatchKey.gsp: 22>, <BatchKey.gsp_id: 23>, <BatchKey.gsp_time_utc: 26>, <BatchKey.gsp_y_osgb: 24>, <BatchKey.gsp_x_osgb: 25>, <BatchKey.gsp_5_min: 30>, <BatchKey.gsp_5_min_time_utc: 31>, <BatchKey.hrvsatellite_x_osgb_fourier: 9>, <BatchKey.pv_x_osgb_fourier: 20>, <BatchKey.gsp_x_osgb_fourier: 28>, <BatchKey.hrvsatellite_y_osgb_fourier: 8>, <BatchKey.pv_y_osgb_fourier: 19>, <BatchKey.gsp_y_osgb_fourier: 27>, <BatchKey.hrvsatellite_time_utc_fourier: 10>, <BatchKey.pv_time_utc_fourier: 21>, <BatchKey.gsp_time_utc_fourier: 29>, <BatchKey.gsp_5_min_tim

In [7]:
np_batch[BatchKey.gsp].shape

(31, 6)

In [8]:
np_batch[BatchKey.gsp_x_osgb_fourier].shape

(31, 1, 8)

In [9]:
np_batch[BatchKey.gsp_5_min].shape


(31, 31)

In [10]:
np_batch[BatchKey.gsp_5_min_time_utc]

array([[1.5999030e+09, 1.5999030e+09, 1.5999030e+09, 1.5999030e+09,
        1.5999030e+09, 1.5999030e+09, 1.5999048e+09, 1.5999048e+09,
        1.5999048e+09, 1.5999048e+09, 1.5999048e+09, 1.5999048e+09,
        1.5999066e+09, 1.5999066e+09, 1.5999066e+09, 1.5999066e+09,
        1.5999066e+09, 1.5999066e+09, 1.5999084e+09, 1.5999084e+09,
        1.5999084e+09, 1.5999084e+09, 1.5999084e+09, 1.5999084e+09,
        1.5999101e+09, 1.5999101e+09, 1.5999101e+09, 1.5999101e+09,
        1.5999101e+09, 1.5999101e+09,           nan],
       [1.6007724e+09, 1.6007724e+09, 1.6007724e+09, 1.6007724e+09,
        1.6007724e+09, 1.6007741e+09, 1.6007741e+09, 1.6007741e+09,
        1.6007741e+09, 1.6007741e+09, 1.6007741e+09, 1.6007759e+09,
        1.6007759e+09, 1.6007759e+09, 1.6007759e+09, 1.6007759e+09,
        1.6007759e+09, 1.6007779e+09, 1.6007779e+09, 1.6007779e+09,
        1.6007779e+09, 1.6007779e+09, 1.6007779e+09, 1.6007796e+09,
        1.6007796e+09, 1.6007796e+09, 1.6007796e+09, 1.6007796

In [12]:
np_batch[BatchKey.gsp_5_min_time_utc_fourier][:, :,  0]

array([[0.56904805, 0.56904805, 0.56904805, 0.56904805, 0.56904805,
        0.56904805, 0.9358613 , 0.9358613 , 0.9358613 , 0.9358613 ,
        0.9358613 , 0.9358613 , 0.9700774 , 0.9700774 , 0.9700774 ,
        0.9700774 , 0.9700774 , 0.9700774 , 0.6595363 , 0.6595363 ,
        0.6595363 , 0.6595363 , 0.6595363 , 0.6595363 , 0.11460166,
        0.11460166, 0.11460166, 0.11460166, 0.11460166, 0.11460166,
               nan],
       [0.56904805, 0.56904805, 0.56904805, 0.56904805, 0.56904805,
        0.9358613 , 0.9358613 , 0.9358613 , 0.9358613 , 0.9358613 ,
        0.9358613 , 0.9700774 , 0.9700774 , 0.9700774 , 0.9700774 ,
        0.9700774 , 0.9700774 , 0.6264281 , 0.6264281 , 0.6264281 ,
        0.6264281 , 0.6264281 , 0.6264281 , 0.07155345, 0.07155345,
        0.07155345, 0.07155345, 0.07155345, 0.07155345,        nan,
               nan],
       [0.56904805, 0.56904805, 0.56904805, 0.56904805, 0.56904805,
        0.56904805, 0.9358613 , 0.9358613 , 0.9358613 , 0.9358613 ,
      

In [13]:
np_batch[BatchKey.gsp_5_min][0]

array([0.18378873, 0.18378873, 0.18378873, 0.18378873, 0.18378873,
       0.18378873, 0.37143427, 0.37143427, 0.37143427, 0.37143427,
       0.37143427, 0.37143427, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.02558792, 0.02558792,
       0.02558792, 0.02558792, 0.02558792, 0.02558792, 0.00996516,
       0.00996516, 0.00996516, 0.00996516, 0.00996516, 0.00996516,
              nan], dtype=float32)